# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import warnings
import re
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
warnings.simplefilter('ignore')

In [ ]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql("SELECT * FROM MessagesCategories", engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

### 2. Write a tokenization function to process your text data

In [15]:
def tokenize(text):
    # Converting everything to lower case
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize words
    tokens = word_tokenize(text)
    
    # normalization word tokens and remove stop words
    normlizer = PorterStemmer()
    stop_words = stopwords.words("english")
    
    normlized = [normlizer.stem(word) for word in tokens if word not in stop_words]
    
    return normlized

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [16]:
pipe = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [17]:
# train and test set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 1)

np.random.seed(13)
pipe.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [40]:
def metrics(ArrayL, ArrayP, col_names):
    """
    inputs:
    ArrayL: real labels.
    ArrayP: predicted labels.
    col_names: list of strings, names for ArrayP fields
       
    outputs:
    data_metrics: accuracy, precision, recall 
    and f1 score for the set of labels from ArrayL and ArrayP
    """
    metrics = []
    
    # Evaluate metrics for each set of labels
    for i in range(len(col_names)):
        accuracy = accuracy_score(ArrayL[:, i], ArrayP[:, i])
        precision = precision_score(ArrayL[:, i], ArrayP[:, i], average='micro')
        recall = recall_score(ArrayL[:, i], ArrayP[:, i], average='micro')
        f1 = f1_score(ArrayL[:, i], ArrayP[:, i], average='micro')
        
        metrics.append([accuracy, precision, recall, f1])
    
    # store metrics
    metrics = np.array(metrics)
    data_metrics = pd.DataFrame(data = metrics, index = col_names, columns = ['Accuracy', 'Precision', 'Recall', 'F1'])
      
    return data_metrics    
     

In [41]:
#test message
msg = ['I need help, my house is on fire']
test_output = pipeline.predict(msg)
print(Y_train.columns.values[(test_output.flatten()==1)])

['related' 'request' 'aid_related' 'other_aid' 'buildings']


In [42]:
# Evaluation metrics for training set
Y_train_pred = pipe.predict(X_train)
col_names = list(Y.columns.values)

print(metrics(np.array(Y_train), Y_train_pred, col_names))

                        Accuracy  Precision    Recall        F1
related                 0.990045   0.990045  0.990045  0.990045
request                 0.986912   0.986912  0.986912  0.986912
offer                   0.998989   0.998989  0.998989  0.998989
aid_related             0.985901   0.985901  0.985901  0.985901
medical_help            0.988377   0.988377  0.988377  0.988377
medical_products        0.992622   0.992622  0.992622  0.992622
search_and_rescue       0.993683   0.993683  0.993683  0.993683
security                0.995300   0.995300  0.995300  0.995300
military                0.995503   0.995503  0.995503  0.995503
child_alone             1.000000   1.000000  1.000000  1.000000
water                   0.994846   0.994846  0.994846  0.994846
food                    0.994896   0.994896  0.994896  0.994896
shelter                 0.993026   0.993026  0.993026  0.993026
clothing                0.998029   0.998029  0.998029  0.998029
money                   0.994947   0.994

In [43]:
# Evaluation metrics for test set
Y_test_pred = pipe.predict(X_test)

metrics0 = metrics(np.array(Y_test), Y_test_pred, col_names)
print(metrics0)

                        Accuracy  Precision    Recall        F1
related                 0.806427   0.806427  0.806427  0.806427
request                 0.886615   0.886615  0.886615  0.886615
offer                   0.994543   0.994543  0.994543  0.994543
aid_related             0.752008   0.752008  0.752008  0.752008
medical_help            0.926785   0.926785  0.926785  0.926785
medical_products        0.953009   0.953009  0.953009  0.953009
search_and_rescue       0.971199   0.971199  0.971199  0.971199
security                0.982265   0.982265  0.982265  0.982265
military                0.964832   0.964832  0.964832  0.964832
child_alone             1.000000   1.000000  1.000000  1.000000
water                   0.957708   0.957708  0.957708  0.957708
food                    0.929362   0.929362  0.929362  0.929362
shelter                 0.928907   0.928907  0.928907  0.928907
clothing                0.986509   0.986509  0.986509  0.986509
money                   0.977414   0.977

In [44]:
# Proportion of columns with label ==1 
# (to understand the distribution of positive labels across different categories )
Y.sum()/len(Y)

related                   0.775032
request                   0.171038
offer                     0.004586
aid_related               0.415144
medical_help              0.079550
medical_products          0.049989
search_and_rescue         0.027477
security                  0.017850
military                  0.032707
child_alone               0.000000
water                     0.063822
food                      0.112029
shelter                   0.088759
clothing                  0.015539
money                     0.022967
missing_people            0.011408
refugees                  0.033351
death                     0.045630
other_aid                 0.131282
infrastructure_related    0.064769
transport                 0.045933
buildings                 0.050974
electricity               0.020390
tools                     0.006026
hospitals                 0.010725
shops                     0.004548
aid_centers               0.011711
other_infrastructure      0.043773
weather_related     

### 6. Improve your model
Use grid search to find better parameters. 

In [49]:
# performance metric to use in grid search scoring
def perform_metrics(Y_T, Y_P):
    """median F1 score for classifiers
    inputs:
    Y_T: arrayL labels
    Y_P: arrayP labels
        
    outputs:
    median F1 score for classifiers
    """
    f1_list = []
    for i in range(np.shape(Y_P)[1]):
        f1 = f1_score(np.array(Y_T)[:, i], Y_P[:, i], average='micro')
        f1_list.append(f1)
        
    score = np.median(f1_list)
    return score

In [ ]:
parameters = {'vect__min_df': [1, 5],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[10, 25], 
              'clf__estimator__min_samples_split':[2, 5, 10]}

scorer = make_scorer(perform_metrics)
cv = GridSearchCV(pipe, param_grid = parameters, scoring = scorer, verbose = 10)

# the best parameters
np.random.seed(74)
T_model = cv.fit(X_train, Y_train)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.9558132484462635, total=  48.5s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.1min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.9584596725288054, total=  47.7s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.1min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.9580806549423894, total=  48.5s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5 


...

[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=1, score=0.9581564584596726, total=  47.1s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  8.0min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=1, score=0.957171012734991, total=  47.3s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5 


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  9.0min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5, score=0.9550553281794755, total=  41.5s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5, score=0.9601273499090358, total=  41.4s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5, score=0.960582171012735, total=  41.3s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=1, score=0.9553584962861907, total= 1.5min
[CV] clf__estimator__min_samples_split=2, clf__e

In [ ]:
#results
T_model.cv_results_

In [ ]:
#max mean score
np.max(T_model.cv_results_['mean_test_score'])

In [ ]:
# Parameters for top mean score
T_model.best_params_

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
# metrics for test set
tuned_p_test = T_model.predict(X_test)

metrics1 = metrics(np.array(Y_test), tuned_pred_test, col_names)

print(metrics1)

In [ ]:
# comparison of model before and after tuning
metrics0.describe()
metrics1.describe()

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# try SVM 
pipe_SVM = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(SVC()))
])

parameters_SVM = {'vect__min_df': [5],
              'tfidf__use_idf':[True],
              'clf__estimator__kernel': ['poly'], 
              'clf__estimator__degree': [1, 2, 3],
              'clf__estimator__C':[1, 10, 100]}

cv_SVM = GridSearchCV(pipe_SVM, param_grid = parameters_SVM, scoring = scorer, verbose = 10)

# Find best parameters
np.random.seed(77)
T_model_SVM = cv_SVM .fit(X_train, Y_train)

In [ ]:
# results
T_model_SVM.cv_results_

In [ ]:
# Calculate evaluation metrics for test set
tuned_p_test2 = T_model_SVM.predict(X_test)

metrics2 = metrics(np.array(Y_test), tuned_p_test2, col_names)

print(metrics2)

### 9. Export your model as a pickle file

In [ ]:
# model with better results
with open('classifier', 'wb') as model_file:
    pickle.dump(T_model, model_file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.